## ■ 시간차 학습 p.105

몬테카를로의 단점을 개선을 한것이 시간차 학습 입니다,  
__몬테카를로 알고리즘의 단점__은, 바둑의 경우처럼 에피소드의 길이가 길거나 에피소드의 끝이 없는 주식시장처럼 에피소드의 끝이 없는경우는   
몬테카를로 예측이 적합하지 않습니다.  

### off policy는 말 그대로 현재 행동하는 정책과는 독립적으로 학습한다는 것입니다.  
### 즉 행동하는 정책과 학습하는 정책을  따로 분리하는것을 말합니다.  
### off policy를 적용한게 큐러닝인데 1989년 chris watkin라는 분에 의해서 소개가 되었습니다.  

## ■ 큐러닝(p.115)

벨만 기대 방정식 --> 정책 반복 --> 살사
벨만 최적 방정식 --> 가치 반복 --> 큐러닝(off policy)

__첨부파일__: Qlearning.pdf

## 문제.(오늘의 마지막 문제) 큐러닝 틱텍토를 살사 틱텍토로 변경하시오~

## ■ 인공지능 틱텍토 프로그램의 큰 그림( p.249)

## 1. Enviroment 클래스 (p.255)

- init 함수 : 게임 초기화
- move 함수 : 플레이어의 선택을 보드에 표시하고 게임이 종료되었는지를 체크
- get_action 함수 : 현재 보드 상태에서 가능한 행동(둘수있는 장소) 선택
- end_check 함수 : 게임이 종료되었는지 판단
- print_board 함수 : 현재 보드의 상태를 출력

## 2. 인간 플레이어 클래스
## 3. 랜덤 플레이어 클래스
## 4. 몬테카를로 플레이어 클래스
## 5. Q-learning 플레이어 클래스
## 6. DQN 플레이어 클래스(신경망)

## ■ 큐러닝 플레이어 클래스를 이해하기 위해서 알아야 할 파이썬 기본 문법

In [51]:
# 1. np.random.choice

import  numpy as  np

# pr의 리스트는 합쳐서 무조건 1이 되어야 한다.
pr= [0.1,0.1,0.8]

action = np.random.choice(range(0,3), p=pr)  # [0,1,2]
print(action)

2


In [58]:
# 2. np,where

import numpy as np
a = np.array([1,2,3,10,20,30,0.1,0.2])

print(np.min(a), np.max(a))
print('\n')
print(a[np.where(a<1)])
print(np.where(a>=10,0,a)) # a numpy 리스트에서 10보다 크거나 같으면 0으로 아니면 a로 출력

0.1 30.0


[0.1 0.2]
[1.  2.  3.  0.  0.  0.  0.1 0.2]


In [66]:
# 3. 딕셔너리 자료형 만들기

dic = {}
dic['a'] = 0.234
dic['b'] = 0.443

print(dic)
print(dic.keys())
print(dic.values())
print(dic.items())

{'a': 0.234, 'b': 0.443}
dict_keys(['a', 'b'])
dict_values([0.234, 0.443])
dict_items([('a', 0.234), ('b', 0.443)])
['a', 'b']


## 틱텍토의 학습데이터가 쌓이는 딕셔너리의 형태

In [67]:
qtable = {}
key = ((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3) # 앞의것은 보드판 상황, 뒤의 3은 3번자리에 둘 것이라는 action
qtable[key] = 0 # 3번자리에 두는것에 대한 가치 (초기 가치라서 0으로 설정)
print(qtable)

{((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3): 0}


(1.0, 1.0, 0.0,   
0.0, -1.0, -1.0,   
-1.0, 0.0, 1.0)  

| o | o |   |
|:-:|:-:|---|
|   | x | x |
| x |   | o |

In [69]:
# 4. tqdm 모듈 사용법
# 루프문의 진행상황을 막대그래프로 표시해줌.

from tqdm import tqdm
from time import sleep

for step in tqdm(range(500)):
    sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:51<00:00,  9.78it/s]


## ■ Q-leanring 클래스 리뷰

In [70]:
# init 참고 코드

qtable = {}
key = ((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3) # 앞의것은 보드판 상황, 뒤의 3은 3번자리에 둘 것이라는 action
qtable[key] = 0 # 3번자리에 두는것에 대한 가치 (초기 가치라서 0으로 설정)
print(qtable)

{((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3): 0}


In [ ]:
class Q_learning_player():
    
    def __init__(self):
        self.name = "Q_player"
        # Q-table을 딕셔너리로 정의
        self.qtable = {} # ((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3) = 0.2912
        # e-greedy 계수 정의
        self.epsilon = 1 # 학습할때 epsilon을 0.5로 하고 실제로 사람과 게임할 때는 랜덤수를 두지 않게 한다.
        
        # 학습률 정의
        self.learning_rate = 0.1
        self.gamma=0.9 # 감가율
        self.print = False

    # policy에 따라 상태에 맞는 행동을 선택
    def select_action(self, env, player):
        # policy에 따라 행동을 선택
        action = self.policy(env) # env : 보드판
    
        return action # 0,1,2,3,4,5,6,7,8 중에 하나가 출력
        
    def policy(self, env):
        # 행동 가능한 상태를 저장
        available_action = env.get_action() # [2,3,4]
        # 행동 가능한 상태의 Q-value를 저장
        qvalues = np.zeros(len(available_action)) # 가치값(Q값) [0,0,0]
        

        if self.print:
            print("{} : available_action".format(available_action))

        # 행동 가능한 상태의 Q-value를 조사
        for i, act in enumerate(available_action): # [2.3.4]

            key = (tuple(env.board_a),act)
            # ((1.0, 1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 1.0), 3) 

            # 현재 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
            if self.qtable.get(key) ==  None:                
                self.qtable[key] = 0
                
            # 행동 가능한 상태의 Q-value 저장 (현재상태를 경험한 적이 있을 경우)
            qvalues[i] = self.qtable.get(key)
     


        # e-greedy
        # 가능한 행동들 중에서 Q-value가 가장 큰 행동을 저장
        greedy_action = np.argmax(qvalues)                    
        
        pr = np.zeros(len(available_action)) # [0,0,0]
      

        # max Q-value와 같은 값이 여러개 있는지 확인한 후 double_check에 상태를 저장
        double_check = (np.where(qvalues == np.max(qvalues),1,0))

        #  여러개 있다면 중복된 상태중에서 다시 무작위로 선택    
        if np.sum(double_check) > 1:
         
            double_check = double_check/np.sum(double_check)
            greedy_action =  np.random.choice(range(0,len(double_check)), p=double_check)
            
        # e-greedy로 행동들의 선택 확률을 계산
        pr = np.zeros(len(available_action))
        
    # epsilon 이 0.5이므로 두번중에 한번은 경험이고 한번은 random이다.
        for i in range(len(available_action)):
            if i == greedy_action:
                pr[i] = 1 - self.epsilon + self.epsilon/len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],2)))
            else:
                pr[i] = self.epsilon / len(available_action)
                if pr[i] < 0:
                    print("{} : - pr".format(np.round(pr[i],2)))

        action = np.random.choice(range(0,len(available_action)), p=pr)

      
        return available_action[action]    # 3,2 등등 random
    
    def learn_qtable(self,board_bakup, action_backup, env, reward):
        # 현재 상태와 행동을 키로 저장
        key = (board_bakup,action_backup)
     
        # Q-table 학습
        if env.done == True:
            # 게임이 끝났을 경우 학습

            self.qtable[key] += self.learning_rate*(reward - self.qtable[key])
            
         
        else:
            # 게임이 진행중일 경우 학습
            # 다음 상태의 max Q 값 계산
            available_action = env.get_action()        
            qvalues = np.zeros(len(available_action))

            for i, act in enumerate(available_action):
                next_key = (tuple(env.board_a),act)
                # 다음 상태를 경험한 적이 없다면(딕셔너리에 없다면) 딕셔너리에 추가(Q-value = 0)
                if self.qtable.get(next_key) ==  None:                
                    self.qtable[next_key] = 0
                qvalues[i] = self.qtable.get(next_key)

            # maxQ 조사
            maxQ = np.max(qvalues)  
            
            # 게임이 진행중일 때 학습
            self.qtable[key] += self.learning_rate*(reward + self.gamma * maxQ - self.qtable[key])
        
                
        if self.print:
            print("-----------   learn_qtable end -------------")